In [27]:
#!/usr/bin/env python
import sys
from pathlib import Path

from ase.io import read, write
from ase.spacegroup import get_spacegroup

def isequal_species(species1:list[str], species2:list[str]):
    """判断species1中全部元素是否都在species2中"""
    if len(species1) == len(species2):
        for spe in species1:
            if spe not in species2:
                return False
        else:
            return True

# species = sys.argv[1:]
species = ['Ce', 'Al']
# 获取当前目录下的所有文件
current_dir = Path('../test')

# 筛选出以 .vasp 结尾的文件名
vasp_files = [file for file  in current_dir.iterdir() if file.suffix == '.vasp']

i = 0
for vf in vasp_files:
    at = read(vf)
    if isequal_species(species1=species, species2=set(at.symbols)):
        i += 1
        newname = current_dir.joinpath(str(i)+'.'+f'{at.symbols}' + '.vasp')
        write(filename=newname, images=at, format='vasp')
    else:
        print(False)


False
False
False
False
False
False
False
